In [1]:
import pandas as pd

In [2]:
from keras.layers import Input, Dense
from snn import SNN, Entity, Relation

Using Theano backend.


In [3]:
Person = Entity("Person")
Movie = Entity("Movie")

In [4]:
df = pd.read_csv("../data/relations.tsv", sep="\t", names=['Movie', 'Relation', 'Person'], nrows=10000)

In [5]:
df.head()

,Movie,Relation,Person
0,Carmencita,self,Carmencita
1,Carmencita,director,William K.L. Dickson
2,Carmencita,cinematographer,William Heise
3,Le clown et ses chiens,director,Émile Reynaud
4,Le clown et ses chiens,composer,Gaston Paulin


In [6]:
relations = {l: Relation(label=l, dst=Person, src=Movie) for l in df['Relation'].unique()}
relations

{'self': self(<Movie> , <Person>),
 'director': director(<Movie> , <Person>),
 'cinematographer': cinematographer(<Movie> , <Person>),
 'composer': composer(<Movie> , <Person>),
 'producer': producer(<Movie> , <Person>),
 'editor': editor(<Movie> , <Person>),
 'actor': actor(<Movie> , <Person>),
 'actress': actress(<Movie> , <Person>),
 'writer': writer(<Movie> , <Person>),
 'production_designer': production_designer(<Movie> , <Person>)}

In [7]:
entities = [Person, Movie]
entities

[<Person>, <Movie>]

In [8]:
import json

In [9]:
ttt = json.load(open('outs/IMDB_ONTO_EMBEDING_NEW_data.json'))
vocab = ttt['vocabulary']
del ttt

In [10]:
X = [t.Person + " @ " + t.Movie for t in df.itertuples()]
y = [[relations[t.Relation]] for t in df.itertuples()]

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
cv = CountVectorizer(vocabulary=vocab)
Xt = cv.fit_transform(X)

In [13]:
x = Input(shape=(Xt.shape[1],))
snn = SNN(entities=entities, relations=relations.values())
output = snn(x)

In [14]:
from snn.utils import draw

In [15]:
snn_model = snn.build(x)

In [16]:
yp = snn_model.predict(Xt)
yp = (yp > 0.5).astype(int)
yp

array([[1, 0, 1, ..., 1, 1, 1],
       [1, 0, 1, ..., 1, 1, 0],
       [1, 0, 1, ..., 1, 1, 0],
       ...,
       [1, 1, 1, ..., 0, 1, 1],
       [0, 1, 1, ..., 0, 1, 1],
       [0, 1, 0, ..., 0, 1, 0]])

In [17]:
yt = snn.map(y)
yt

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
snn_model.fit(Xt, yt, epochs=10, validation_split=0.1)

Train on 9000 samples, validate on 1000 samples
Epoch 1/10
9000/9000 [==============================] - 33s 4ms/step - loss: 0.1371 - val_loss: 0.1305
Epoch 2/10
9000/9000 [==============================] - 34s 4ms/step - loss: 0.0615 - val_loss: 0.1247
Epoch 3/10
9000/9000 [==============================] - 29s 3ms/step - loss: 0.0478 - val_loss: 0.1246
Epoch 4/10
9000/9000 [==============================] - 21s 2ms/step - loss: 0.0401 - val_loss: 0.1312
Epoch 5/10
9000/9000 [==============================] - 23s 3ms/step - loss: 0.0354 - val_loss: 0.1408
Epoch 6/10
9000/9000 [==============================] - 23s 3ms/step - loss: 0.0322 - val_loss: 0.1612
Epoch 7/10
9000/9000 [==============================] - 23s 3ms/step - loss: 0.0294 - val_loss: 0.1744
Epoch 8/10
9000/9000 [==============================] - 24s 3ms/step - loss: 0.0280 - val_loss: 0.1818
Epoch 9/10
9000/9000 [==============================] - 26s 3ms/step - loss: 0.0263 - val_loss: 0.1790
Epoch 10/10
9000/9000 [==

In [19]:
yp = snn_model.predict(Xt)
yp = (yp > 0.5).astype(int)
(yp == yt).mean()

0.9894833333333334

In [20]:
snn_model.save('resources/imdb_onto.model')

In [21]:
import pickle

In [22]:
pickle.dump(cv, open('resources/imdb_onto_cv.pkl', 'wb'))